In [2]:
import cv2
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Fungsi untuk ekstraksi histogram dari gambar
def extract_histogram(image, bins=(8, 8, 8)):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# Fungsi untuk memuat gambar dan label dari folder
def load_images_from_folder(folder):
    data = []
    labels = []
    for class_label in os.listdir(folder):
        class_path = os.path.join(folder, class_label)
        if os.path.isdir(class_path):
            for image_name in os.listdir(class_path):
                image_path = os.path.join(class_path, image_name)
                image = cv2.imread(image_path)
                if image is not None:
                    hist = extract_histogram(image)
                    data.append(hist)
                    labels.append(class_label)
    return np.array(data), np.array(labels)

# Folder gambar untuk training dan testing
train_dir = "../images/training/"
test_dir = "../images/test/"

# Muat data training dan testing
X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# Normalisasi data dengan StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Inisialisasi model SVM
svm_model = SVC(kernel='linear')

# Latih model SVM
svm_model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = svm_model.predict(X_test)

# Evaluasi performa
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:")
print(report)


Accuracy: 97.50%
Classification Report:
              precision    recall  f1-score   support

         day       0.96      0.99      0.98        80
       night       0.99      0.96      0.97        80

    accuracy                           0.97       160
   macro avg       0.98      0.98      0.97       160
weighted avg       0.98      0.97      0.97       160

